In [1]:
from datetime import date, datetime, timedelta
import pymysql
import csv
import os
from pandas import DataFrame
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector
from mysql.connector import errorcode
import glob
import logging

### Configure log information 

In [2]:
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

### Connect to mysql database

In [3]:
try:
    cnn = mysql.connector.connect(
    user='root',
    password='root',
    host='127.0.0.1',
    database='Amazon_review',
    port='3307')
    print ("Amazon_review connection works!")
    cursor = cnn.cursor()
    
except mysql.connector.Error as e:
    if e.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with Username or Password")
    elif e.errno == errorcode.ER_BAD_DB_ERROR:
        print("database Does not exist")
    else:
        print(e)

cursor.execute("show tables;")
results = cursor.fetchall()
logging.info('***********************************************')
logging.info(results)

Amazon_review connection works!


In [4]:
cursor.execute("select product_id, rating,helpful_votes, review_date, title, review from helpful_review;")
amazon_full = cursor.fetchall()
num_fields = len(cursor.description)
field_names = [i[0] for i in cursor.description]
df = pd.DataFrame(amazon_full, columns=field_names)   

In [21]:
t = 'January 5, 2020'.replace(",", "")
t = t.replace(" ", "-")
print(t)
date_object = datetime.strptime(t, "%B-%d-%Y")
date_object


January-5-2020


datetime.datetime(2020, 1, 5, 0, 0)

### Format the data 

In [23]:
df['rating']=pd.to_numeric(df['rating'])
df['helpful_votes']=pd.to_numeric(df['helpful_votes'])
df['review_date']=df['review_date'].apply(lambda x: datetime.strptime(x.replace(",", "").replace(" ", "-"), "%B-%d-%Y"))
#df['review_date'] = df['review_date'].apply(lambda x: )
df.describe(include='all')

,product_id,rating,helpful_votes,review_date,title,review
count,2484,2484.000000,2484.000000,2484,2484,2484
unique,14,NaN,NaN,941,1858,2456
top,B07H7Q1DFP,NaN,NaN,2020-01-05 00:00:00,Five Stars,Great
freq,746,NaN,NaN,14,120,7
first,NaN,NaN,NaN,2008-07-19 00:00:00,NaN,NaN
last,NaN,NaN,NaN,2020-01-25 00:00:00,NaN,NaN
mean,NaN,4.489533,0.464573,NaN,NaN,NaN
std,NaN,1.053680,2.697076,NaN,NaN,NaN
min,NaN,1.000000,0.000000,NaN,NaN,NaN
25%,NaN,4.000000,0.000000,NaN,NaN,NaN


In [24]:
df.review_date.value_counts()

2020-01-05    14
2019-12-12    12
2019-07-09    11
2019-08-23    11
2019-09-18    11
2019-12-14    11
2019-06-24    10
2019-07-22    10
2019-12-17    10
2019-12-26    10
2019-12-16    10
2019-07-17    10
2019-12-21     9
2019-10-08     9
2019-02-24     9
2019-12-18     9
2019-10-04     9
2020-01-11     9
2019-06-19     9
2019-07-24     9
2020-01-03     9
2019-12-27     9
2019-07-08     9
2019-11-18     9
2020-01-12     8
2019-09-17     8
2020-01-21     8
2019-12-15     8
2019-11-14     8
2019-12-31     8
              ..
2017-08-23     1
2018-04-28     1
2017-01-12     1
2019-01-01     1
2017-09-01     1
2018-04-17     1
2017-10-30     1
2018-07-19     1
2017-05-14     1
2016-11-26     1
2017-03-21     1
2018-08-31     1
2018-10-25     1
2017-03-29     1
2017-10-06     1
2018-07-03     1
2018-06-10     1
2018-06-12     1
2016-08-26     1
2018-10-18     1
2017-07-19     1
2015-09-12     1
2015-06-03     1
2018-08-29     1
2017-09-23     1
2018-12-01     1
2016-08-04     1
2017-04-09    